In [1]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/BigDataProject/

/content/drive/MyDrive/BigDataProject


In [3]:
cd data

/content/drive/MyDrive/BigDataProject/data


In [4]:
ls

data_0_10000.csv        data_160000_170000.csv  data_60000_70000.csv
data_100000_110000.csv  data_170000_180000.csv  data_70000_80000.csv
data_10000_20000.csv    data_180000_190000.csv  data_80000_90000.csv
data_110000_120000.csv  data_190000_200000.csv  data_90000_100000.csv
data_120000_130000.csv  data_20000_30000.csv    data.csv
data_130000_140000.csv  data_30000_40000.csv    local.json
data_140000_150000.csv  data_40000_50000.csv    url_failed_to_load.txt
data_150000_160000.csv  data_50000_60000.csv


### Merge Data


In [ ]:
import os
import csv

file_names = os.listdir(os.getcwd())
content = []
key = True
for i, file_name in enumerate(file_names):
  l = []
  if file_name[-3:] == 'csv':
    with open(file_name, 'r') as f:
      reader = csv.reader(f)
      l = list(reader)
  if key == True and file_name[-3:] == 'csv':
    content.append(l[0])
    key = False
  content.extend(l[1:])


In [ ]:
len(content)

199555

In [ ]:
with open('data.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(content)

### Preprocess

In [5]:
import pandas as pd
df = pd.read_csv('./data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199554 entries, 0 to 199553
Data columns (total 24 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Tên             199554 non-null  object
 1   Mô tả           199401 non-null  object
 2   Mức giá         199363 non-null  object
 3   Diện tích       197105 non-null  object
 4   Loại tin đăng   199368 non-null  object
 5   Địa chỉ         199347 non-null  object
 6   Mặt tiền        92019 non-null   object
 7   Đường vào       87346 non-null   object
 8   Hướng ban công  26589 non-null   object
 9   Số tầng         68211 non-null   object
 10  Số phòng ngủ    94068 non-null   object
 11  Số toilet       79611 non-null   object
 12  Nội thất        47865 non-null   object
 13  Pháp lý         113105 non-null  object
 14  Tên dự án       72643 non-null   object
 15  Chủ đầu tư      58496 non-null   object
 16  Quy mô          53555 non-null   object
 17  Ngày đăng       199368 non-nu

In [7]:
df.head(5)

,Tên,Mô tả,Mức giá,Diện tích,Loại tin đăng,Địa chỉ,Mặt tiền,Đường vào,Hướng ban công,Số tầng,Số phòng ngủ,Số toilet,Nội thất,Pháp lý,Tên dự án,Chủ đầu tư,Quy mô,Ngày đăng,Ngày hết hạn,Mã tin,Phòng ngủ,Hướng nhà,Loại tin,url
0,Chính chủ cần bán gấp lô đất ngay gần chợ Sông...,"- Ngay chợ Sông Trầu.- Dân cư đang phát triển,...",860 triệu,90 m²,Bán đất,"Đường Sông Trầu, Xã Sông Trầu, Trảng Bom, Đồng...",NaN,8 m,NaN,NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN,NaN,NaN,06/11/2021,13/11/2021,31319056,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...
1,"Bán lô đất TĐC giai đoạn 3 Bửu Long, Biên Hòa ...","Bán lô đất TĐC giai đoạn 3 Bửu Long, Biên Hòa....",2.2 tỷ,79.8 m²,Bán đất,"Phường Bửu Long, Biên Hòa, Đồng Nai",NaN,10 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2021,13/11/2021,31319055,NaN,Nam,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...
2,Duy nhất căn nhà phố vị trí đẹp khu Habor 1 Aq...,* Nhà phố giá tốt - đẹp không khác gì shophous...,15.419 tỷ,138 m²,"Bán nhà biệt thự, liền kề (nhà trong dự án quy...","Dự án Aqua City, Đường Hương lộ 2, Xã Long Hưn...",NaN,NaN,NaN,4 tầng,NaN,NaN,NaN,NaN,Aqua City,Novaland Group,"11 phân khu với 15000 căn nhà phố, biệt thự, s...",06/11/2021,16/11/2021,31319054,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...
3,"Nhà mặt tiền đường Lê Hồng Phong, ngay vòng xo...","Bán nhà mặt tiền đường Lê Hồng Phong, phường P...",6.5 tỷ,74.5 m²,Bán nhà mặt phố (nhà mặt tiền trên các tuyến phố),"Đường Lê Hồng Phong, Phường Phước Long, Nha Tr...",4.77 m,30 m,NaN,3 tầng,4 phòng,3 phòng,NaN,Sổ đỏ/ Sổ hồng,NaN,NaN,NaN,06/11/2021,13/11/2021,31319053,4 PN,Tây-Nam,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...
4,"Nhà MT đường số 4 Linh Xuân, ngay chợ 3MT đườn...","Nhà MT đường số 4, Linh Xuân, ngay chợ 3MT đườ...",1.8 tỷ,80 m²,Bán nhà mặt phố (nhà mặt tiền trên các tuyến phố),"Đường Số 4, Phường Linh Xuân, Thủ Đức, Hồ Chí ...",4 m,20 m,NaN,1 tầng,2 phòng,2 phòng,Cơ bản,Mua bán công chứng vi bằng,NaN,NaN,NaN,06/11/2021,13/11/2021,31319052,2 PN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...


In [8]:
### Drop invalid rows
df = df[df['url'].notna()]

In [9]:
key = 'Mã tin'
df[key] = df[key].fillna(-1).astype('int64')

In [10]:
### Drop exception
errors = []
for index, col in zip(df.index, df['Loại tin đăng']):
  if 'Cho thuê' in col:
    errors.append(index)
df.drop(errors, inplace=True)

In [11]:
set(df['Loại tin đăng'])

{'Bán condotel',
 'Bán căn hộ chung cư',
 'Bán kho, nhà xưởng',
 'Bán loại bất động sản khác',
 'Bán nhà biệt thự, liền kề (nhà trong dự án quy hoạch)',
 'Bán nhà mặt phố (nhà mặt tiền trên các tuyến phố)',
 'Bán nhà riêng',
 'Bán trang trại, khu nghỉ dưỡng',
 'Bán đất',
 'Bán đất nền dự án (đất trong dự án quy hoạch)'}

In [18]:
import re
new_df = {}
float_col = ['Mặt tiền','Đường vào']
int_col = ['Số tầng','Số phòng ngủ','Số toilet']
for col in float_col:
  new_df[col] = []
  for u in df[col]:
    try:
      u = u.replace(',','.')
      v = re.findall(r'[-+]?\d*\.*\d+', u)
      if len(v)!=1:
        print(u)
      new_df[col].append(float(v[0]))
      #new_df[col].append(int(v[0]))
    except:
      new_df[col].append(-1)
      if str(u)!='nan':
        print(u)
      continue
for col in int_col:
  new_df[col] = []
  for u in df[col]:
    try:
      u = u.replace(',','.')
      v = re.findall(r'[-+]?\d*\.*\d+', u)
      if len(v)!=1:
        print(u)
      #new_df[col].append(float(v[0]))
      new_df[col].append(int(v[0]))
    except:
      new_df[col].append(-1)
      if str(u)!='nan':
        print(u)
      continue

In [20]:
for key in new_df.keys():
  df[key] = new_df[key]

In [ ]:
df.columns

Index(['Tên', 'Mô tả', 'Mức giá', 'Diện tích', 'Loại tin đăng', 'Địa chỉ',
       'Mặt tiền', 'Đường vào', 'Hướng ban công', 'Số tầng', 'Số phòng ngủ',
       'Số toilet', 'Nội thất', 'Pháp lý', 'Tên dự án', 'Chủ đầu tư', 'Quy mô',
       'Ngày đăng', 'Ngày hết hạn', 'Mã tin', 'Phòng ngủ', 'Hướng nhà',
       'Loại tin', 'url'],
      dtype='object')

Mức giá và diện tích

In [ ]:
currency_convert = {
    'nghìn': 1.0/1000, 'nghìn/m²': 1.0/1000, 'thuận': 'Thỏa thuận', 'triệu': 1, 'triệu/m²': 1, 'tỷ': 1000, 'tỷ/m²': 1000, 'đồng/m²': 1.0/1000000
}
test = []
for price, s in zip(df['Mức giá'], df['Diện tích']):
  u = price.strip().split()
  if 'm²' in u[1]:
    if str(s)=='nan':
      test.append(None)
      continue
    area = float(s.strip().split()[0])
    p = float(u[0])*currency_convert[u[1]]
    test.append(area*p)
  else:
    if u[1]!='thuận':
      test.append(float(u[0])*currency_convert[u[1]])
    else:
      test.append(None)

In [ ]:
new_df['Mức giá'] = test

In [ ]:
new_df['Diện tích'] = []
for s in df['Diện tích']:
  try:
    u = s.strip().split()
    new_df['Diện tích'].append(float(u[0]))
  except:
    new_df['Diện tích'].append(None)

Địa chỉ

In [ ]:
add_df = pd.read_csv('../Danh sách quận huyện Việt Nam 2021 Full - ©DiaOcThongThai.Com.csv')
add_df.head(5)

,STT,Tỉnh Thành,Quận Huyện
0,1,Hà Nội,Quận Ba Đình
1,2,Hà Nội,Quận Bắc Từ Liêm
2,3,Hà Nội,Quận Cầu Giấy
3,4,Hà Nội,Quận Đống Đa
4,5,Hà Nội,Quận Hà Đông


In [ ]:
re.sub('-',' ','Bà Rịa-Vũng Tàu')

'Bà Rịa Vũng Tàu'

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
similar('Quận Long Biên', 'Long Biên')

0.782608695652174

In [ ]:
#quan in re.sub('\W+',' ',add_df['Quận Huyện'][i]):
def check_add(add_df, tinh, quan):
  tinh = re.sub('\W+',' ',tinh).strip()
  quan = re.sub('\W+',' ',quan).strip()
  for i in range(add_df.shape[0]):
    if tinh in re.sub('\W+',' ',add_df['Tỉnh Thành'][i]) or similar(tinh, add_df['Tỉnh Thành'][i])>0.5:
      if similar(quan, add_df['Quận Huyện'][i])>0.5 or quan in re.sub('\W+',' ',add_df['Quận Huyện'][i]):
        return True, add_df['Tỉnh Thành'][i], add_df['Quận Huyện'][i]
  return False, '', ''

In [ ]:
a = pd.DataFrame({
    'STT': [708],
    'Tỉnh Thành': ['Hồ Chí Minh','Hồ Chí Minh'],
    'Quận Huyện': ['Quận 2','Quận 9']
})
add_df = add_df.append(a, ignore_index=True)

In [ ]:
add_df[add_df['Tỉnh Thành']=='Gia Lai']

,STT,Tỉnh Thành,Quận Huyện
261,262,Gia Lai,Thành phố Pleiku (tỉnh lỵ)
262,263,Gia Lai,Thị xã An Khê
263,264,Gia Lai,Thị xã Ayun Pa
264,265,Gia Lai,Huyện Chư Păh
265,266,Gia Lai,Huyện Chư Prông
266,267,Gia Lai,Huyện Chư Pưh
267,268,Gia Lai,Huyện Chư Sê
268,269,Gia Lai,Huyện Đak Đoa
269,270,Gia Lai,Huyện Đak Pơ
270,271,Gia Lai,Huyện Đức Cơ


In [ ]:
similar(' Hà Nội',' Hà Nội')

0.625

In [ ]:
error_index = []
ds_tinh = []
ds_quan = []
errors = []
for i in df.index:
  address = str(df['Địa chỉ'][i])
  address = address.replace('-',',')
  devided_address = address.split(',')

  if len(devided_address)<2:
    print(address)
    ds_tinh.append(None)
    ds_quan.append(None)
    continue
  if ' Hà Nội' in devided_address[-1]:
    devided_address[-1] = 'Hà Nội'
  check, tinh, quan = check_add(add_df, devided_address[-1].strip(), devided_address[-2].strip())
  if not check:
    errors.append(address)
    ds_tinh.append(None)
    ds_quan.append(None)
    continue
    #error_index.append(i)
  ds_tinh.append(tinh)
  ds_quan.append(quan)
#df['Tỉnh'] = ds_tinh
#df['Quận'] = ds_quan
#df = df.drop(error_index,axis=0)

Đường Nguyễn Khuyến Phường Văn Quán Quận Hà Đông   Hà Nội
Phố Việt Hưng Phường Việt Hưng Quận Long Biên   Hà Nội
Phố Việt Hưng Phường Việt Hưng Quận Long Biên   Hà Nội
112 minh phung
Đường Thạch Bàn Phường Thạch Bàn Quận Long Biên   Hà Nội
Đường Thạch Bàn Phường Thạch Bàn Quận Long Biên   Hà Nội
112 minh phung
112 minh phung
112 minh phung
112 minh phung
Siêu víp 110m 5T 22 tỷ
Đường Xô Viết Nghệ Tĩnh Phường 21 Quận Bình Thạnh   Hồ Chí Minh
Đường Xô Viết Nghệ Tĩnh Phường 21 Quận Bình Thạnh   Hồ Chí Minh
Phố Ngõ Trạm Phường Long Biên Quận Long Biên   Hà Nội
Văn Giang
Văn Giang
Đường Tỉnh Lộ 422B Xã Vân Canh Huyện Hoài Đức   Hà Nội Xã Vân Canh Huyện Hoài Đức   Hà Nội
Văn Giang
KHU CÔNG NGHIỆP ĐÔNG MAI QUẢNG YÊN QUẢNG NINH
327/5 Nguyễn Văn Khối(Cây Trâm) P.8 Q. Gò Vấp Tp.HCM
Đường Võ Văn Kiệt phường Cầu Kho Quận 1 Hồ Chí Minh
Đường Thạch Bàn Phường Thạch Bàn Quận Long Biên   Hà Nội


In [ ]:
for key in new_df.keys():
  if len(new_df[key])<2:
    new_df[key] = list(df[key])

In [ ]:
pd.DataFrame(new_df).to_csv('../data_preprocessed.csv', index=False)

In [ ]:
df = pd.read_csv('../data_preprocessed.csv')

In [ ]:
len(df)

199323

In [ ]:
df = df.drop(columns=['Địa chỉ'])

In [ ]:
df.iloc[[226,227,228,229,230]]

,Tên,Mô tả,Mức giá,Diện tích,Loại tin đăng,Địa chỉ,Mặt tiền,Đường vào,Hướng ban công,Số tầng,Số phòng ngủ,Số toilet,Nội thất,Pháp lý,Tên dự án,Chủ đầu tư,Quy mô,Ngày đăng,Ngày hết hạn,Mã tin,Phòng ngủ,Hướng nhà,Loại tin,url
226,"Bán BT Hoa Phượng 03 - 06 hướng ĐN, 298.1m2 đấ...",Thông tin lô biệt thự siêu đẹp khu Hoa Phượng ...,25.5 tỷ,298.1 m²,"Bán nhà biệt thự, liền kề (nhà trong dự án quy...","Dự án Vinhomes Riverside, Đường Hoa Phượng 3, ...",NaN,NaN,NaN,3 tầng,NaN,NaN,NaN,NaN,Vinhomes Riverside,Tập đoàn Vingroup,3000 căn biệt thự,06/11/2021,13/11/2021,15609400,NaN,Đông-Nam,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...
227,Bán nhà HXH căn góc 2 mặt tiền Bùi Quang là 4....,Bán nhà HXH căn góc 2 mặt tiền Bùi Quang là 4....,6.9 tỷ,82 m²,Bán nhà riêng,"Đường Bùi Quang Là, Phường 12, Gò Vấp, Hồ Chí ...",NaN,6 m,NaN,1 tầng,4 phòng,4 phòng,NaN,Sổ đỏ chính chủ,NaN,NaN,NaN,06/11/2021,16/11/2021,30807159,4 PN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...
228,Chính chủ bán lỗ 5 tỷ Vinpearl PQ2 2T4PN - giá...,- Chủ nhà cần bán nhanh căn biệt thự mặt biển ...,18 tỷ,450 m²,"Bán nhà biệt thự, liền kề (nhà trong dự án quy...","Dự án Vinpearl Phú Quốc, Gành Dầu, Phú Quốc, K...",NaN,NaN,NaN,2 tầng,4 phòng,NaN,NaN,NaN,Vinpearl Phú Quốc,Tập đoàn Vingroup,NaN,06/11/2021,13/11/2021,31219222,4 PN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...
229,"Cần bán gấp đất hẻm 8m Phước Kiển, Lê Văn Lươn...","Cần bán gấp đất hẻm 8m Phước Kiển, Lê Văn Lươn...",35.5 triệu/m²,126 m²,Bán đất,"Đường Lê Văn Lương, Xã Phước Kiển, Nhà Bè, Hồ ...",NaN,8 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2021,13/11/2021,31225082,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...
230,"Bán căn hộ Duplex Happy Valley, hỗ trợ trả góp...",- Bán căn hộ Penthouse phong cách resort 5 sao...,Thỏa thuận,179 m²,Bán căn hộ chung cư,"Dự án Happy Valley, Đường Nguyễn Văn Linh, Phư...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Happy Valley,Công ty TNHH Phát Triển Phú Mỹ Hưng,NaN,06/11/2021,13/11/2021,23688847,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-can...


In [ ]:
df.head(10)

,Tên,Mô tả,Mức giá,Diện tích,Loại tin đăng,Địa chỉ,Mặt tiền,Đường vào,Hướng ban công,Số tầng,Số phòng ngủ,Số toilet,Nội thất,Pháp lý,Tên dự án,Chủ đầu tư,Quy mô,Ngày đăng,Ngày hết hạn,Mã tin,Phòng ngủ,Hướng nhà,Loại tin,url
0,Chính chủ cần bán gấp lô đất ngay gần chợ Sông...,"- Ngay chợ Sông Trầu.- Dân cư đang phát triển,...",860 triệu,90 m²,Bán đất,"Đường Sông Trầu, Xã Sông Trầu, Trảng Bom, Đồng...",NaN,8 m,NaN,NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN,NaN,NaN,06/11/2021,13/11/2021,31319056,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...
1,"Bán lô đất TĐC giai đoạn 3 Bửu Long, Biên Hòa ...","Bán lô đất TĐC giai đoạn 3 Bửu Long, Biên Hòa....",2.2 tỷ,79.8 m²,Bán đất,"Phường Bửu Long, Biên Hòa, Đồng Nai",NaN,10 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2021,13/11/2021,31319055,NaN,Nam,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...
2,Duy nhất căn nhà phố vị trí đẹp khu Habor 1 Aq...,* Nhà phố giá tốt - đẹp không khác gì shophous...,15.419 tỷ,138 m²,"Bán nhà biệt thự, liền kề (nhà trong dự án quy...","Dự án Aqua City, Đường Hương lộ 2, Xã Long Hưn...",NaN,NaN,NaN,4 tầng,NaN,NaN,NaN,NaN,Aqua City,Novaland Group,"11 phân khu với 15000 căn nhà phố, biệt thự, s...",06/11/2021,16/11/2021,31319054,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...
3,"Nhà mặt tiền đường Lê Hồng Phong, ngay vòng xo...","Bán nhà mặt tiền đường Lê Hồng Phong, phường P...",6.5 tỷ,74.5 m²,Bán nhà mặt phố (nhà mặt tiền trên các tuyến phố),"Đường Lê Hồng Phong, Phường Phước Long, Nha Tr...",4.77 m,30 m,NaN,3 tầng,4 phòng,3 phòng,NaN,Sổ đỏ/ Sổ hồng,NaN,NaN,NaN,06/11/2021,13/11/2021,31319053,4 PN,Tây-Nam,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...
4,"Nhà MT đường số 4 Linh Xuân, ngay chợ 3MT đườn...","Nhà MT đường số 4, Linh Xuân, ngay chợ 3MT đườ...",1.8 tỷ,80 m²,Bán nhà mặt phố (nhà mặt tiền trên các tuyến phố),"Đường Số 4, Phường Linh Xuân, Thủ Đức, Hồ Chí ...",4 m,20 m,NaN,1 tầng,2 phòng,2 phòng,Cơ bản,Mua bán công chứng vi bằng,NaN,NaN,NaN,06/11/2021,13/11/2021,31319052,2 PN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...
5,Chính chủ cần bán nhanh lô đất mặt tiền nhựa B...,Chính chủ cần bán nhanh lô đất mặt tiền nhựa B...,Thỏa thuận,125 m²,Bán đất,"Đường Bắc Hòa - Phú Sơn, Xã Bắc Sơn, Trảng Bom...",5 m,6 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2021,16/11/2021,30500121,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...
6,"Cần bán nhà hẻm xe hơi Huỳnh Mẫn Đạt, diện tíc...","Bán giảm giá mùa dịch nhà HXH Huỳnh Mẫn Đạt, Q...",11 tỷ,77 m²,Bán nhà riêng,"Đường Huỳnh Mẫn Đạt, Phường 2, Quận 5, Hồ Chí ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2021,13/11/2021,30920576,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...
7,"Chính chủ cần bán đất P28, Bình Thạnh, SHR, 75...","Chính chủ cần bán gấp 1 lô duy nhất, đất mặt t...",4.6 tỷ,75 m²,Bán đất,"Đường Bình Quới, Phường 28, Bình Thạnh, Hồ Chí...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Có sổ,NaN,NaN,NaN,06/11/2021,13/11/2021,19662900,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...
8,Cơ hội đầu tư TP Thủ Đức! Đất KDC Hiệp Bình mớ...,"Mở bán đợt 1 mặt tiền đường Hiệp Bình, phường ...",2.8 tỷ,75 m²,Bán đất,"Đường Hiệp Bình, Phường Hiệp Bình Phước, Thủ Đ...",5 m,8 m,NaN,NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng.,NaN,NaN,NaN,06/11/2021,13/11/2021,31319051,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...
9,"Đất đẹp khu trung tâm thương mại 550, gần Viet...","Cần bán đất đường D3, kinh doanh buôn bán.Khu ...",5 tỷ,100 m²,Bán đất,"Đường D3, Phường Dĩ An, Dĩ An, Bình Dương",5 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2021,14/11/2021,31319050,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...


Save to csv

In [ ]:
df.to_csv('preprocessed_data.csv')

In [ ]:
pd.read_csv('preprocessed_data.csv').head(10)

,Unnamed: 0,Tên,Mô tả,Mức giá,Diện tích,Loại tin đăng,Mặt tiền,Đường vào,Hướng ban công,Số tầng,Số phòng ngủ,Số toilet,Nội thất,Pháp lý,Tên dự án,Chủ đầu tư,Quy mô,Ngày đăng,Ngày hết hạn,Mã tin,Phòng ngủ,Hướng nhà,Loại tin,url,Tỉnh,Quận
0,0,Chính chủ cần bán gấp lô đất ngay gần chợ Sông...,"- Ngay chợ Sông Trầu.- Dân cư đang phát triển,...",860.0,90.0,Bán đất,NaN,8.0,NaN,NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN,NaN,NaN,06/11/2021,13/11/2021,31319056.0,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...,Đồng Nai,Huyện Trảng Bom
1,5,Chính chủ cần bán nhanh lô đất mặt tiền nhựa B...,Chính chủ cần bán nhanh lô đất mặt tiền nhựa B...,-1.0,125.0,Bán đất,5.00,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2021,16/11/2021,30500121.0,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...,Đồng Nai,Huyện Trảng Bom
2,6,"Cần bán nhà hẻm xe hơi Huỳnh Mẫn Đạt, diện tíc...","Bán giảm giá mùa dịch nhà HXH Huỳnh Mẫn Đạt, Q...",11000.0,77.0,Bán nhà riêng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2021,13/11/2021,30920576.0,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...,Hồ Chí Minh,Quận 5
3,7,"Chính chủ cần bán đất P28, Bình Thạnh, SHR, 75...","Chính chủ cần bán gấp 1 lô duy nhất, đất mặt t...",NaN,75.0,Bán đất,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Có sổ,NaN,NaN,NaN,06/11/2021,13/11/2021,19662900.0,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...,Hồ Chí Minh,Quận Bình Thạnh
4,10,"Bán gấp căn hộ cao cấp Fortuna Kim Hồng, Quận ...",Bán gấp căn hộ cao cấp Fortuna Kim Hồng Quận T...,NaN,75.0,Bán căn hộ chung cư,NaN,NaN,NaN,NaN,2.0,2.0,NaN,Sổ hồng,Fortuna - Vườn Lài,Công ty TNHH Dịch vụ Thương mại Kim Hồng,NaN,06/11/2021,13/11/2021,31319048.0,2.0,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-can...,Hồ Chí Minh,Quận Tân Phú
5,11,"Chính chủ bán mảnh đất mặt ngõ 38 Tư Đình, Lon...","- Chủ bán mảnh đất ngõ 38 Tư Đình, Long Biên, ...",NaN,76.0,Bán đất,4.06,4.0,NaN,4.0,3.0,3.0,NaN,Sổ đỏ.,NaN,NaN,NaN,06/11/2021,13/11/2021,30957020.0,3.0,Tây-Bắc,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...,Hà Nội,Quận Long Biên
6,14,Chính chủ bán nhà xây mới ngõ 108 phố Đông Thi...,"Chính chủ cần bán nhà ngõ 108 phố Đông Thiên, ...",NaN,37.6,Bán nhà riêng,4.00,NaN,NaN,5.0,3.0,4.0,NaN,Sổ đỏ/ Sổ hồng.,NaN,NaN,NaN,06/11/2021,13/11/2021,31319046.0,3.0,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...,Hà Nội,Quận Hoàng Mai
7,15,Nhà mặt đường Hàng Kênh - Lê Chân - Hải Phòng,"Nhà mặt đường Hàng Kênh.- DTMB: 54m2, ngang 4,...",NaN,54.0,Bán nhà mặt phố (nhà mặt tiền trên các tuyến phố),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN,NaN,NaN,06/11/2021,13/11/2021,31226161.0,NaN,NaN,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...,Hải Phòng,Quận Lê Chân
8,16,"Bán gấp biệt thự Cổ Nhuế - Nam Cường, 3.5 tầng...",Cần bán căn biệt thự xây thô tại khu Nam Cường...,117.0,225.0,"Bán nhà biệt thự, liền kề (nhà trong dự án quy...",10.00,10.0,Đông-Nam,4.0,4.0,6.0,NaN,Sổ hồng chính chủ,Khu đô thị mới Cổ Nhuế,Công ty CP Tập đoàn Nam Cường Hà Nội,Gồm khối văn phòng cho thuê và 3 tòa chung cư ...,06/11/2021,13/11/2021,30557861.0,4.0,Đông-Nam,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-nha...,Hà Nội,Quận Bắc Từ Liêm
9,17,"Bán đất Hà Đông 3 tại quận Thanh Khê, Đà Nẵng:...",Cảm ơn quý khách đã quan tâm lô đất đường Hà Đ...,54.0,97.0,Bán đất,5.00,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2021,13/11/2021,30931085.0,NaN,Tây,Tin thường,https://batdongsan.com.vn/nha-dat-ban//ban-dat...,Đà Nẵng,Quận Thanh Khê
